<a href="https://colab.research.google.com/github/RubenCarton623/Bot/blob/main/BotDefinitivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip es un sistema de gestión de paquetes utilizado para instalar y administrar paquetes de software escritos en Python.
!apt update  
!apt install chromium-chromedriver
!pip install selenium

!pip install python-telegram-bot-calendar
!pip install pyTelegramBotAPI

In [ ]:
from telegram_bot_calendar import DetailedTelegramCalendar, LSTEP
import telebot
from datetime import date
from telebot import types
from bs4 import BeautifulSoup
from time import sleep
import time
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')#sin cabecera
options.add_argument('--no-sandbox')#deshabilitar caja de arena
options.add_argument('--disable-dev-shm-usage')#deshabilitar el uso de dev shm

Verificacion_cedula = True

IATA = {"Guayaquil" : "GYE", "Cuenca" : "CUE", "Manta" : "MEC",
		"Coca" : "OCC", "Santa Rosa" : "ETR", "Quito" : "UIO",
		"Baltra" : "GPS", "San Cristobal" : "SCY"}#Lista de palabras reservadas para los vuelos

CLASES = {"Economico":"NA", "Premium Economico":"PE", "Ejecutivo":"C", "Primera Clase":"F"}#lista de palabras reservadas para las clases de vuelos

bot = telebot.TeleBot("1632069612:AAE80oO5RmGWKDFFjGVevz4WFicFjhC18l4")#token del bot

vuelo_dict = {}

class Vuelo:
    def __init__(self, Cliente):
        self.Cliente = Cliente
        self.Origen = None
        self.Destino = None
        self.Adultos = None
        self.Niños = None
        self.Salida = None
        self.Retorno = None
        self.Tipo = None
        self.Clase = None
        self.EdadN = None
        self.Horas = None
        self.Duracion = None
        self.Escalas = None
        self.HorasR = None
        self.DuracionR = None
        self.EscalasR = None
        self.Precio = None
        self.Oferta = None
        self.HoraSalida = None
        self.HoraRetorno = None
        self.PrecioV = None
        self.Reservas = None

In [ ]:
def Crear_Url(Tipo, Salida, Adultos, Niños, Clase, Origen=None, Destino=None, EdadN = None, Retorno=None):

    if Destino and Origen:#Si hay origen y hay destino
        url = ("https://www.despegar.com.ec/shop/flights/results/{}/{}/{}/{}").format(Tipo,Origen,Destino,Salida)

    if Retorno:#Si hay retorno
        url=url+"/"+str(Retorno)

    url=url+"/"+Adultos #Agg adultos 

    if Niños != 0:#Sii hay niños
        url=url+"/"
        for i in range(Niños):
            if i == Niños-1:
                url = url + "C"
            else:
                url = url + "C-"#C-C-C sig que hay 3 niños
    else:
        url = url + "/0"#si no hay niños entonces en la url se pone /0

    if not Retorno:#Si no hay retorno se arma la url SIN retorno
        url = url + "/0/NA/{}/NA/NA?from=SB&di={}-{}".format(Clase, Adultos, Niños)
    else:#url sin retorno
        url=url+"/0/NA/{}/NA/NA/NA?from=SB&di={}-{}".format(Clase, Adultos, Niños)

    textoEdad=""
    if EdadN:#si hay edad de los niños
        for i in range(len(EdadN)):
            if i == len(EdadN)-1:
                textoEdad = textoEdad + EdadN[i]
            else:
                textoEdad = textoEdad + EdadN[i] + "-"

    if Niños != 0:#si hay niños se agregan sus edades
        url = url + ":" + textoEdad

    url = url + "&reSearch=true"#esto lleva al final de la url

    return url

In [ ]:
def vuelosOrigenSalida(Origen=None, Destino=None):
    if Origen: 
        url = "https://www.flightradar24.com/data/airports/{}/{}".format(Origen, "departures")
    elif Destino:
        url = "https://www.flightradar24.com/data/airports/{}/{}".format(Destino, "arrivals")

    driver = webdriver.Chrome('chromedriver',options=options)#crear una nueva instancia de la clase ChromeDriver
    driver.get(url)#le damos la url a la pag creada
    #scraping
    soup = BeautifulSoup(driver.page_source, 'lxml')
    vuelos = soup.find_all('td', attrs={'class': 'state-block-gray'})#scrap de los vuelos con estatus Scheduled

    limite = 0 #ponemos un limite para que se scrapee 6 veces en caso de que no se haya cargado la pag
    while len(vuelos) < 1:  #mientras no se hayan encontrado vuelos se recorre 6 veces
        sleep(1)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        vuelos = soup.find_all('td', attrs={'class': 'state-block-gray'})
        limite = limite + 1
        if limite == 5:
            break

    driver.close()

    Origenes=list()
    Destinos=list()
    Horas=list()
    Aerolineas=list()
    Aeropuertos=list()

    for i in range(0,len(vuelos)):
        aeropuerto = vuelos[i].findChildren('a', attrs={'class': 'notranslate ng-binding'})
        for j in aeropuerto:
            if j.text.find("(")!= -1: 
                if str(IATA).find(j.text.replace("(", "").replace(")", ""))!=-1:
                    hora = vuelos[i].findChildren('span', attrs={'class': 'ng-binding'})
                    aerolinea = vuelos[i].findChildren('a', attrs={'ng-show': '(objFlight.flight.airline.name)'})

                    if Origen:
                        Origenes.append(Origen)
                        Destinos.append(j.text.replace("(", "").replace(")", ""))
                    elif Destino:
                        Origenes.append(j.text.replace("(", "").replace(")", ""))
                        Destinos.append(Destino)

                    for y in hora:
                        if y.text.find(":")!=-1:
                            Horas.append(y.text)

                    for y in  aerolinea:
                        Aerolineas.append(y.text)

    return Origenes, Destinos, Horas, Aerolineas

In [ ]:
def VuelosProgramados():
    Aeropuertos = IATA.values()
    Origenes = list()
    Destinos = list()
    Horas = list()
    Aerolineas = list()

    for i in Aeropuertos:
        OrigenesT, DestinosT, HorasT, AerolineasT = vuelosOrigenSalida(i)

        Origenes.extend(OrigenesT)
        Destinos.extend(DestinosT)
        Horas.extend(HorasT)
        Aerolineas.extend(AerolineasT)

    return Origenes, Destinos, Horas, Aerolineas

In [ ]:
def Scraping(Tipo, Salida, Adultos, Niños, Clase, Origen=None, Destino=None, EdadN = None, Retorno=None):
    url = (Crear_Url(Tipo, Salida, Adultos, Niños, Clase, Origen, Destino, EdadN, Retorno))

    print(url)
    driver = webdriver.Chrome('chromedriver',options=options)
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'lxml')
    Ofertas = soup.find_all('div', attrs={'class': 'cluster-container COMMON'})

    limite=0
    while len(Ofertas)<1:
        sleep(2)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        Ofertas = soup.find_all('div', attrs={'class': 'cluster-container COMMON'})
        limite=limite+1
        if limite==5:
            break

    driver.close()

    HorasS = list()
    DuracionS = list()
    EscalasS = list()
    PrecioF = list()

    if Retorno:
        HorasR = list()
        DuracionR = list()
        EscalasR = list()
        PrecioR = list()

    for i in Ofertas:
        Precio = i.findChildren('p', attrs={'class': 'item-fare fare-price'})
        if not Retorno:
            Ida = i.findChildren('span', attrs={'class': 'sub-cluster last'})
        else:


            Ida = [el for el in i.findChildren('span', class_='sub-cluster') if 'last' not in el.attrs['class']]
            Vuelta = i.findChildren('span', attrs={'class': 'sub-cluster last'})


        for j in Ida:
            Horas = j.findChildren('span', attrs={'class': 'hour'})
            Duracion = j.findChildren('span', attrs={'class': 'best-duration' , 'class': 'duration-item-container'})
            Escalas = j.findChildren('span', attrs={'data-sfa-id': 'stops-text'})


            HorasT = list()
            for j in Horas:
                HorasT.append(j.text)

            DuracionT = list()
            for j in Duracion:
                DuracionT.append(j.text)

            EscalasT = list()
            for j in Escalas:
                if "escala" in j.text.lower():
                    if "escalas" in j.text.lower():
                        EscalasT.append(j.text.replace(" ","").replace("escalas", ""))
                    else:
                        EscalasT.append(j.text.replace(" ", "").replace("escala", ""))
                elif "directo" in j.text.lower():
                    EscalasT.append("0")

            HorasS.append(HorasT)
            DuracionS.append(DuracionT)
            EscalasS.append(EscalasT)

        if Retorno:
            for j in Vuelta:
                Horas = j.findChildren('span', attrs={'class': 'hour'})
                Duracion = j.findChildren('span', attrs={'class': 'best-duration'})
                Duracion2 = j.findChildren('span', attrs={'class': 'duration-item-container'})
                Escalas = j.findChildren('span', attrs={'data-sfa-id': 'stops-text'})


                HorasT = list()
                for j in Horas:
                    HorasT.append(j.text)

                DuracionT = list()
                for j in Duracion:
                    DuracionT.append(j.text)

                for j in Duracion2:
                    DuracionT.append(j.text)

                EscalasT = list()
                for j in Escalas:
                    if "escala" in j.text.lower():
                        if "escalas" in j.text.lower():
                            EscalasT.append(j.text.replace(" ","").replace("escalas", ""))
                        else:
                            EscalasT.append(j.text.replace(" ", "").replace("escala", ""))
                    elif "directo" in j.text.lower():
                        EscalasT.append("0")

                HorasR.append(HorasT)
                DuracionR.append(DuracionT)
                EscalasR.append(EscalasT)

        PrecioT = 0
        for j in Precio:
            valor = j.text.replace("Precio final USD", "").replace(".", "")
            if int(valor):
                PrecioT = str(valor)
        PrecioF.append(PrecioT)


    if not Retorno:
        return HorasS, DuracionS, EscalasS, PrecioF
    else:
        return HorasS, DuracionS, EscalasS, PrecioF, HorasR, DuracionR, EscalasR,

In [ ]:
def listener(messages):

    for idx, m in enumerate(messages):

        Chat_ID = m.chat.id
        if Chat_ID in vuelo_dict:
            vuelo = vuelo_dict[Chat_ID]

        if Chat_ID > 0:
            mensaje = str(m.chat.first_name) + " [" + str(Chat_ID) + "]: " + m.text

        else:
            mensaje = str(m.from_user.first_name) + "[" + str(Chat_ID) + "]: " + m.text
        print(mensaje)  # Imprimimos el mensaje en la terminal, que nunca viene mal :)

        if m.text.lower().find("menu") != -1 and Chat_ID in vuelo_dict and vuelo.id != "":

            for i in range(0, int(m.message_id) - int(vuelo.id)):
                try:
                    bot.delete_message(Chat_ID, m.message_id - i)
                except:
                    pass
            vuelo.id = ""




@bot.message_handler(regexp='Menu|start')
def Menu(message):
    try:
        chat_id = message.chat.id



        if chat_id not in vuelo_dict:
            vuelo = Vuelo(bot.get_chat(chat_id).first_name)
            vuelo_dict[chat_id] = Vuelo
            vuelo = vuelo_dict[chat_id]
            vuelo.id=""
            vuelo.Reservas = list()


        vuelo = vuelo_dict[chat_id]
        vuelo.id =""

        vuelo.Salida = ""
        vuelo.Retorno = ""
        vuelo.Origen = ""
        vuelo.Destino = ""
        vuelo.Niños = ""
        vuelo.EdadN = ""
        vuelo.Nombres = list()
        vuelo.Cedulas = list()
        vuelo.EdadN = list()

        bot.send_message(chat_id, "----- 𝐌𝐄𝐍𝐔 -----")
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Buscar Vuelos", "Listar Vuelos", "Mis reservas")
        pregunta = bot.send_message(chat_id, "Selecciona o digita una de las opciones del menu:", reply_markup=markup)
        bot.register_next_step_handler(pregunta, TipoVuelo)

    except Exception as e:
        bot.send_message(chat_id, 'Ops, algo no salio bien... ¡Intentalo de nuevo!')


def TipoVuelo(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]


        vuelo.id = ""

        opcion = message.text
        if opcion.lower().find("buscar vuelos") == -1 and opcion.lower().find("mis reservas") == -1 and opcion.lower().find(
                "listar vuelos") == -1:
            bot.send_message(chat_id, "----- 𝐌𝐄𝐍𝐔 -----")
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Buscar Vuelos", "Listar Vuelos", "Mis reservas")
            pregunta = bot.send_message(chat_id, "Selecciona o digita una de las opciones del menu:",
                                        reply_markup=markup)
            bot.register_next_step_handler(pregunta, TipoVuelo)
            return

        if opcion.lower().find("buscar vuelos") != -1:
            bot.send_message(chat_id, "----- 𝐁𝐔𝐒𝐂𝐀𝐑 𝐕𝐔𝐄𝐋𝐎 -----")
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Ida", "Ida y vuelta", "Cancelar")
            pregunta = bot.send_message(chat_id, 'Seleccione o digite el tipo de vuelo:', reply_markup=markup)

            bot.register_next_step_handler(pregunta, Origen)
        elif opcion.lower().find("mis reservas") != -1:
            bot.send_message(chat_id, '----- 𝐌𝐈𝐒 𝐑𝐄𝐒𝐄𝐑𝐕𝐀𝐒 -----')
            for i in vuelo.Reservas:
                datos = i.split("|")

                pasajeros = int(datos[2])
                Nombres="\nPasajeros:"
                for i in range(len(datos)-pasajeros, len(datos)):
                    Nombres = Nombres + "\n- " + datos[i]

                if len(datos)-pasajeros > 7:
                    reserva = list(IATA.keys())[list(IATA.values()).index(datos[0])] + " - " + list(IATA.keys())[list(IATA.values()).index(datos[1])] + "\nAsientos: " + datos[
                        2] + Nombres + "\nSalida: " + datos[3] + "\nHora: " + datos[5] \
                              + "\nRetorno: " + datos[4] + "\nHora: " + datos[6] + "\nCosto total: " + datos[7]
                else:
                    reserva = list(IATA.keys())[list(IATA.values()).index(datos[0])] + " - " + list(IATA.keys())[list(IATA.values()).index(datos[1])] + "\nAsientos: " + datos[
                        2] + Nombres + "\nSalida: " + datos[3] + "\nHora: " + datos[4] + "\nCosto total: " + datos[5]

                bot.send_message(chat_id, reserva)

            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Volver al menu")
            pregunta = bot.send_message(chat_id, 'Seleccione o digite menu:', reply_markup=markup)

        elif opcion.lower().find("listar vuelos") != -1:
            bot.send_message(chat_id, '----- 𝐋𝐈𝐒𝐓𝐀𝐑 𝐕𝐔𝐄𝐋𝐎𝐒 -----')
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Origen", "Destino", "Todos", "Cancelar")
            pregunta = bot.send_message(chat_id, 'Seleccione bajo que criterio quiere listar los vuelos:',
                                        reply_markup=markup)

            bot.register_next_step_handler(pregunta, OrigenDestino)

    except Exception as e:
        bot.send_message(chat_id, 'Ops, algo no salio bien... ¡Intentalo de nuevo!')


def Origen(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]


        opcion = message.text.lower()
        if opcion == "ida":
            opcion = "oneway"
        elif opcion == "ida y vuelta":
            opcion = "roundtrip"

        if opcion != "oneway" and opcion != "roundtrip" and opcion != "cancelar":
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Ida", "Ida y vuelta","Cancelar")
            pregunta = bot.send_message(chat_id, "¡Seleccione una tipo de vuelo valido!", reply_markup=markup)
            bot.register_next_step_handler(pregunta, Origen)
            return

        if opcion == "cancelar":
            bot.send_message(chat_id, "----- 𝐌𝐄𝐍𝐔 -----")
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Buscar Vuelos", "Listar Vuelos", "Mis reservas")
            pregunta = bot.send_message(chat_id, "Selecciona o digita una de las opciones del menu:",
                                        reply_markup=markup)
            bot.register_next_step_handler(pregunta, TipoVuelo)
            return

        vuelo.Tipo = opcion

        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal", "Volver")
        pregunta = bot.send_message(chat_id, 'Seleccione o digite el lugar de origen:', reply_markup=markup)

        bot.register_next_step_handler(pregunta, Destino)
    except Exception as e:
        bot.send_message(chat_id, 'Ops, algo no salio bien... ¡Intentalo de nuevo!')


def OrigenDestino(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]

        opcion = message.text.lower()

        if opcion=="cancelar":
            bot.send_message(chat_id, "----- 𝐌𝐄𝐍𝐔 -----")
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Buscar Vuelos", "Listar Vuelos", "Mis reservas")
            pregunta = bot.send_message(chat_id, "Selecciona o digita una de las opciones del menu:",
                                        reply_markup=markup)
            bot.register_next_step_handler(pregunta, TipoVuelo)
            return

        if opcion != "origen" and opcion != "destino" and opcion != "todos":
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Origen", "Destino", "Todos", "Cancelar")
            pregunta = bot.send_message(chat_id, "¡Seleccione una opcion valida!", reply_markup=markup)
            bot.register_next_step_handler(pregunta, OrigenDestino)
            return

        if opcion == "origen":
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal")
            pregunta = bot.send_message(chat_id, 'Seleccione o digite el origen:', reply_markup=markup)
            vuelo.Origen = opcion
            bot.register_next_step_handler(pregunta, ListarOrigenDestino)
        elif opcion == "destino":
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal")
            pregunta = bot.send_message(chat_id, 'Seleccione o digite el destino:', reply_markup=markup)
            vuelo.Destino = opcion
            bot.register_next_step_handler(pregunta, ListarOrigenDestino)
        elif opcion == "todos":

            buscando = bot.send_message(chat_id, 'Buscando vuelos... (Esto puede tardar maximo 2 minutos)')

            Origenes, Destinos, Horas, Aerolineas = VuelosProgramados()
            bot.delete_message(buscando.chat.id, buscando.message_id)
            bot.send_message(chat_id, 'Vuelos programados:')

            if len(Origenes) > 0:
                for i in range(0, len(Origenes)):
                    mensaje = list(IATA.keys())[list(IATA.values()).index(Origenes[i])] + " - " + list(IATA.keys())[list(IATA.values()).index(Destinos[i])] + "\nFecha: " + str(time.strftime("%d/%m/%y")) + "\nHora: " + str(
                        Horas[i]) + "\nAerolinea: " + Aerolineas[i]
                    bot.send_message(chat_id, mensaje)

            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Volver al menu")
            bot.send_message(chat_id, '¿Desea volver al menu?', reply_markup=markup)

    except Exception as e:
        bot.send_message(chat_id, 'Ops, algo no salio bien... ¡Intentalo de nuevo!')


def ListarOrigenDestino(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]

        opcion = message.text

        if str(IATA).find(opcion) == -1:
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal")
            pregunta = bot.send_message(chat_id, '¡Digite una ubicacion valida!', reply_markup=markup)
            bot.register_next_step_handler(pregunta, ListarOrigenDestino)
            return

        buscando = bot.send_message(chat_id, 'Buscando vuelos...')

        if len(vuelo.Origen) > 0:
            Origenes, Destinos, Horas, Aerolineas = vuelosOrigenSalida(IATA.get(opcion))
        else:
            Origenes, Destinos, Horas, Aerolineas = vuelosOrigenSalida(None, IATA.get(opcion))

        bot.delete_message(buscando.chat.id, buscando.message_id)
        if len(Origenes) > 0:
            bot.send_message(chat_id, "Se han programado los siguientes vuelos:")
            for i in range(0, len(Origenes)):
                mensaje = list(IATA.keys())[list(IATA.values()).index(Origenes[i])] + " - " + list(IATA.keys())[list(IATA.values()).index(Destinos[i])] + "\nFecha: " + str(time.strftime("%d/%m/%y")) + "\nHora: " + str(
                    Horas[i]) + "\nAerolinea: " + Aerolineas[i]
                bot.send_message(chat_id, mensaje)

            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Volver al menu")
            bot.send_message(chat_id, '¿Desea volver al menu?', reply_markup=markup)
        else:
            bot.send_message(chat_id, "El aeropuerto seleccionado no tiene vuelos nacionales programados.")
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Volver al menu")
            bot.send_message(chat_id, '¿Desea volver al menu?', reply_markup=markup)
    except Exception as e:
        bot.send_message(chat_id, 'Ops, algo no salio bien... ¡Intentalo de nuevo!')


def Destino(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]

        origen = message.text

        if origen == "Volver":
            bot.send_message(chat_id, "----- 𝐁𝐔𝐒𝐂𝐀𝐑 𝐕𝐔𝐄𝐋𝐎 -----")
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Ida", "Ida y vuelta", "Cancelar")
            pregunta = bot.send_message(chat_id, 'Seleccione o digite el tipo de vuelo:', reply_markup=markup)
            vuelo.Tipo=""
            bot.register_next_step_handler(pregunta, Origen)
            return

        if str(IATA).find(origen) == -1:
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal", "Volver")
            pregunta = bot.send_message(chat_id, '¡Digite un origen valido!', reply_markup=markup)
            bot.register_next_step_handler(pregunta, Destino)
            return

        vuelo.Origen = IATA.get(origen)

        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal", "Volver")
        pregunta = bot.send_message(chat_id, 'Seleccione o digite el lugar de destino:', reply_markup=markup)

        bot.register_next_step_handler(pregunta, Adultos)
    except Exception as e:
        bot.send_message(chat_id, 'Ops, algo no salio bien... ¡Intentalo de nuevo!')


def Adultos(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]
        destino = message.text

        if destino=="Volver":
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal", "Volver")
            pregunta = bot.send_message(chat_id, 'Seleccione o digite el lugar de origen:', reply_markup=markup)
            vuelo.Origen=""
            bot.register_next_step_handler(pregunta, Destino)
            return

        if str(IATA).find(destino) == -1:
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal", "Volver")
            pregunta = bot.send_message(chat_id, '¡Digite un destino valido!', reply_markup=markup)
            bot.register_next_step_handler(pregunta, Adultos)
            return

        if destino.lower() == list(IATA.keys())[list(IATA.values()).index(vuelo.Origen)].lower():
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal", "Volver")
            pregunta = bot.send_message(chat_id, '¡Digite un destino diferente a el origen!', reply_markup=markup)
            bot.register_next_step_handler(pregunta, Adultos)
            return

        vuelo.Destino = IATA.get(destino)

        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("1", "2", "3", "4", "5", "6", "7", "8", "Volver")
        pregunta = bot.send_message(chat_id, 'Seleccione o digite cuantos adultos van a viajar:', reply_markup=markup)

        bot.register_next_step_handler(pregunta, Niños)
    except Exception as e:
        bot.send_message(chat_id, 'Ops, algo no salio bien... ¡Intentalo de nuevo!')

def DatosAdultos(message):
    chat_id = message.chat.id
    message_id = message.message_id
    vuelo = vuelo_dict[chat_id]

    texto = message.text



    if len(vuelo.Nombres)==len(vuelo.Cedulas) or len(vuelo.Nombres)==0:
        vuelo.Nombres.append(texto)
        pregunta = bot.send_message(chat_id, 'Ingrese su cedula:')
        bot.register_next_step_handler(pregunta, DatosAdultos)
        return


    if len(vuelo.Nombres)!=len(vuelo.Cedulas):
        if verificarCedula(texto):
            vuelo.Cedulas.append(texto)
            if len(vuelo.Cedulas)!=int(vuelo.Adultos):
                pregunta = bot.send_message(chat_id, 'Ingrese el nombre del ' + str(len(vuelo.Cedulas)+1) + "° adulto:")
                bot.register_next_step_handler(pregunta, DatosAdultos)
                return
            else:
                markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
                for i in range(9 - int(vuelo.Adultos)):
                    markup.add(str(i))
                markup.add("Volver")
                pregunta = bot.send_message(chat_id, 'Seleccione o digite cuantos niños van a viajar:',
                                            reply_markup=markup)
                vuelo.id = message.message_id + 2
                bot.register_next_step_handler(pregunta, Fecha)
                return
        else:
            pregunta = bot.send_message(chat_id, '¡Ingrese una cedula valida!')
            bot.register_next_step_handler(pregunta, DatosAdultos)
            return

def Niños(message):
    try:

        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]
        adultos = message.text
        vuelo.id = message.message_id + 2

        if adultos == "Volver":
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal", "Volver")
            pregunta = bot.send_message(chat_id, 'Seleccione o digite el lugar de destino:', reply_markup=markup)
            vuelo.Destino=""
            bot.register_next_step_handler(pregunta, Adultos)
            return

        if not adultos.isdigit():
            if adultos == "+12 (Tasa de adulto)":
                adultos = "12"
            else:
                adultos = "0"

        if int(adultos) < 1 or int(adultos) > 8:
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("1", "2", "3", "4", "5", "6", "7", "8", "Volver")
            pregunta = bot.send_message(chat_id, 'Seleccione o digite una cantidad valida:', reply_markup=markup)
            bot.register_next_step_handler(pregunta, Niños)
            return

        vuelo.Adultos = adultos

        pregunta = bot.send_message(chat_id, 'Ingrese el nombre de el ' + str(len(vuelo.Cedulas)+1) + "° adulto:")
        bot.register_next_step_handler(pregunta, DatosAdultos)

    except Exception as e:
        bot.send_message(chat_id, 'Ops, algo no salio bien... ¡Intentalo de nuevo!')


def Fecha(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]
        niños = message.text
        vuelo.id = message.message_id + 2
        if niños=="Volver":
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("1", "2", "3", "4", "5", "6", "7", "8", "Volver")
            pregunta = bot.send_message(chat_id, 'Seleccione o digite cuantos adultos van a viajar:',
                                        reply_markup=markup)
            vuelo.Adultos=""
            bot.register_next_step_handler(pregunta, Niños)
            return

        if not niños.isdigit():
            niños = "10"

        if int(niños) < 0 or int(niños) > 9 - int(vuelo.Adultos):

            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            for i in range(8 - int(vuelo.Adultos)):
                markup.add(str(i))
            markup.add("Volver")
            pregunta = bot.send_message(chat_id, 'Seleccione o digite una cantidad valida:', reply_markup=markup)
            bot.register_next_step_handler(pregunta, Fecha)
            return

        vuelo.Niños = niños
        vuelo.EdadN = list()

        if niños == "0":
            calendar, step = DetailedTelegramCalendar(min_date=date.today(),
                                                      max_date=date(date.today().year, date.today().month,
                                                                    date.today().day)).build()



            if vuelo.Salida == "":
                bot.send_message(message.chat.id, "Seleccione la fecha de salida:", reply_markup=calendar)
            else:
                bot.send_message(message.chat.id, "Seleccione la fecha de retorno:", reply_markup=calendar)

        else:

            pregunta = bot.send_message(chat_id, 'Nombre del 1° niño: ')

            bot.register_next_step_handler(pregunta, seleccionarEdades)

    except Exception as e:
        bot.send_message(chat_id, 'Ops, algo no salio bien... ¡Intentalo de nuevo!')


def seleccionarEdades(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]

        edad = message.text
        vuelo.id = message.message_id + 2

        if len(vuelo.EdadN)==len(vuelo.Nombres)-int(vuelo.Adultos):
            vuelo.Nombres.append(edad)
            pregunta = bot.send_message(chat_id, "Ingrese su cedula:")
            bot.register_next_step_handler(pregunta, seleccionarEdades)
            return

        if len(vuelo.EdadN)==len(vuelo.Cedulas)-int(vuelo.Adultos):
            if verificarCedula(edad):
                vuelo.Cedulas.append(edad)
                markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
                markup.add("1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "+12 (Tasa de adulto)")
                pregunta = bot.send_message(chat_id, "Edad del niño:",
                                            reply_markup=markup)
                bot.register_next_step_handler(pregunta, seleccionarEdades)
                return
            else:
                pregunta = bot.send_message(chat_id, "¡Ingrese una cedula valida!")
                bot.register_next_step_handler(pregunta, seleccionarEdades)
                return

        if not edad.isdigit():
            if edad.find("12") != -1:
                edad = "12"
            else:
                edad = "0"

        if int(edad) < 1 or int(edad) > 12:
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "+12 (Tasa de adulto)")
            pregunta = bot.send_message(chat_id, "Edad del  niño:",
                                        reply_markup=markup)
            bot.register_next_step_handler(pregunta, seleccionarEdades)
            return

        vuelo.EdadN.append(edad)

        if len(vuelo.EdadN) == int(vuelo.Niños):

            texto = "Niños: " + vuelo.Niños + "\n"
            for i in vuelo.EdadN:
                if int(i) == 1:
                    texto = texto + "- Tarifa bebé\n"
                if int(i) == 12:
                    texto = texto + "- Tarifa adulto\n"
                if int(i) < 12 and int(i) > 1:
                    texto = texto + "- Tarifa niño\n"

            bot.send_message(chat_id, texto)

            calendar, step = DetailedTelegramCalendar(min_date=date.today(),
                                                      max_date=date(date.today().year, date.today().month,
                                                                    date.today().day)).build()

            if vuelo.Salida == "":
                bot.send_message(message.chat.id, "Seleccione la fecha de salida:", reply_markup=calendar)
            else:
                bot.send_message(message.chat.id, "Seleccione la fecha de retorno:", reply_markup=calendar)
        else:
            pregunta = bot.send_message(chat_id, 'Nombre del ' + str(len(vuelo.EdadN) + 1) + "° niño:")
            bot.register_next_step_handler(pregunta, seleccionarEdades)

    except Exception as e:
        bot.send_message(chat_id, 'Ops, algo no salio bien... ¡Intentalo de nuevo!')


def Clase(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]
        clase = message.text
        vuelo.id = message.message_id

        if str(CLASES).find(clase) == -1:
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Economico", "Premium Economico", "Ejecutivo", "Primera Clase")
            pregunta = bot.send_message(chat_id, '¡Digite una clase valida!', reply_markup=markup)
            bot.register_next_step_handler(pregunta, Clase)
            return

        pregunta = bot.send_message(chat_id, 'Estoy buscando los mejores vuelos para ti...')


        vuelo.id = message.message_id
        BuscarVuelo(message)
    except Exception as e:
        pregunta = bot.send_message(chat_id, 'Ops, algo no salio bien... ¡Intentalo de nuevo!')


def NoVuelos(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]

        opcion = message.text

        if "cambiar fecha|cambiar clase|buscar otro vuelo|volver al menu".find(opcion.lower()) == -1:
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Cambiar fecha", "Cambiar clase", "Buscar otro vuelo", "Volver al menu")
            pregunta = bot.send_message(chat_id, '¡Seleccione una opcion valida!', reply_markup=markup)
            bot.register_next_step_handler(pregunta, NoVuelos)
            return

        if opcion.lower() == "cambiar fecha":
            vuelo.Salida = ""
            vuelo.Retorno = ""
            calendar, step = DetailedTelegramCalendar(min_date=date.today(),
                                                      max_date=date(date.today().year, date.today().month,
                                                                    date.today().day)).build()
            if vuelo.Salida == "":
                bot.send_message(message.chat.id, "Seleccione la fecha de salida:", reply_markup=calendar)
            else:
                bot.send_message(message.chat.id, "Seleccione la fecha de retorno:", reply_markup=calendar)

        elif opcion.lower() == "cambiar clase":
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Economico", "Premium Economico", "Ejecutivo", "Primera Clase")
            pregunta = bot.send_message(chat_id, 'Seleccione en que clase desea viajar:', reply_markup=markup)

            bot.register_next_step_handler(pregunta, Clase)

        elif opcion.lower() == "buscar otro vuelo":
            vuelo.Origen = ""
            vuelo.Destino = ""
            vuelo.Salida = ""
            vuelo.Retorno = ""
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal")
            pregunta = bot.send_message(chat_id, 'Seleccione o digite el lugar de origen:', reply_markup=markup)

            bot.register_next_step_handler(pregunta, Destino)

        elif opcion.lower() == "volver al menu":
            bot.send_message(chat_id, "----- 𝐌𝐄𝐍𝐔 -----")
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Buscar Vuelos", "Listar Vuelos", "Mis reservas")
            pregunta = bot.send_message(chat_id, "Selecciona o digita una de las opciones del menu:",
                                        reply_markup=markup)
            bot.register_next_step_handler(pregunta, TipoVuelo)

    except Exception as e:
        bot.send_message(chat_id, 'Ops, algo no salio bien... ¡Intentalo de nuevo!')


def BuscarVuelo(message):
    chat_id = message.chat.id
    message_id = message.message_id
    vuelo = vuelo_dict[chat_id]
    vuelo.id = message.message_id
    clase = message.text
    vuelo.Clase = CLASES.get(clase)



    if vuelo.Retorno == "":
        vuelo.Horas, vuelo.Duracion, vuelo.Escalas, vuelo.Precio = Scraping(vuelo.Tipo, vuelo.Salida, vuelo.Adultos,
                                                                            int(vuelo.Niños), vuelo.Clase, vuelo.Origen,
                                                                            vuelo.Destino, vuelo.EdadN)
    else:
        vuelo.Horas, vuelo.Duracion, vuelo.Escalas, vuelo.Precio, vuelo.HorasR, vuelo.DuracionR, vuelo.EscalasR = Scraping(
            vuelo.Tipo, vuelo.Salida, vuelo.Adultos, int(vuelo.Niños), vuelo.Clase, vuelo.Origen, vuelo.Destino,
            vuelo.EdadN, vuelo.Retorno)

    if len(vuelo.Precio) > 0:
        bot.delete_message(chat_id, message_id + 1)
        bot.send_message(chat_id, "¡Listo! He encontrado los siguientes vuelos:")

        for i in range(0, len(vuelo.Precio)):
            message = "*\-\-\-\- Salida \-\-\-\-*\n"
            markup = types.InlineKeyboardMarkup()
            markup_button = types.InlineKeyboardButton("Reservar",
                                                       callback_data=str(i) + "|" + str(vuelo.Precio[i]))
            markup.row(markup_button)

            contador = 0
            for j in range(0, len(vuelo.Escalas[i])):
                message = message + "*" + vuelo.Origen + "* _" + vuelo.Horas[i][
                    contador] + "_ \- *" + vuelo.Destino + "* _" + \
                          vuelo.Horas[i][contador + 1] + "_\n*\- Duracion:* _" + vuelo.Duracion[i][
                              j] + "_ " + "*\n\- Escalas:* _" + vuelo.Escalas[i][j]  + "_\n"
                contador = contador + 2

            if vuelo.Retorno != "":
                message = message + "*\-\-\-\- Retorno \-\-\-\-*\n"
                contador = 0
                for j in range(0, len(vuelo.EscalasR[i])):
                    message = message + "*" + vuelo.Origen + "* _" + vuelo.HorasR[i][
                        contador] + "_ \- *" + vuelo.Destino + "* _" + \
                              vuelo.HorasR[i][contador + 1] + "_\n*\- Duracion:* _" + vuelo.DuracionR[i][
                                  j] + "_ " + "*\n\- Escalas:* _" + vuelo.EscalasR[i][j] + "_\n"
                    contador = contador + 2

            message = message + "*Costo Final:* $" + vuelo.Precio[i]
            bot.send_message(chat_id, message, parse_mode="MarkdownV2", reply_markup=markup,
                             disable_web_page_preview=True)

        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Volver al menu")
        bot.send_message(chat_id, '¿Desea volver al menu?', reply_markup=markup)

    else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Cambiar fecha", "Cambiar clase", "Buscar otro vuelo", "Volver al menu")
        pregunta = bot.send_message(chat_id,
                                    'No se encontraron vuelos en la fecha o clase seleccionada... ¿Que deberiamos hacer?',
                                    reply_markup=markup)

        bot.register_next_step_handler(pregunta, NoVuelos)


def HoraSalida(message):
    chat_id = message.chat.id
    message_id = message.message_id
    vuelo = vuelo_dict[chat_id]
    i = vuelo.Oferta

    salida = message.text
    valida = False
    if salida.find("-") != -1:
        salidaT = salida.replace("-", "").replace(":", "").replace(" ", "")
        if salidaT.isdigit() and len(salidaT) == 8:
            valida = True

    if valida == False:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)

        contador = 0
        for j in range(0, len(vuelo.Escalas[i])):
            markup.add(vuelo.Horas[i][contador] + " - " + vuelo.Horas[i][contador + 1])
            contador = contador + 2

        pregunta = bot.send_message(chat_id, "Seleccione la hora de salida:", reply_markup=markup)
        bot.register_next_step_handler(pregunta, HoraSalida)
        return

    vuelo.HoraSalida = salida

    if vuelo.Retorno != "":
        contador = 0
        for j in range(0, len(vuelo.EscalasR[i])):
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add(vuelo.HorasR[i][contador] + " - " + vuelo.HorasR[i][contador + 1])
            contador = contador + 2

        pregunta = bot.send_message(chat_id, "Seleccione la hora de retorno:", reply_markup=markup)
        bot.register_next_step_handler(pregunta, HoraRetorno)
    else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Si", "No")
        pregunta = bot.send_message(chat_id, '¿Esta seguro de que quiere reservar el vuelo?', reply_markup=markup)

        bot.register_next_step_handler(pregunta, Reservar)


def Reservar(message):
    chat_id = message.chat.id
    message_id = message.message_id
    vuelo = vuelo_dict[chat_id]

    opcion = message.text

    if opcion.lower() != "si" and opcion.lower() != "no":
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Si", "No")
        pregunta = bot.send_message(chat_id, '¿Esta seguro de que quiere reservar el vuelo?', reply_markup=markup)

        bot.register_next_step_handler(pregunta, Reservar)
        return

    if opcion.lower() == "si":
        pregunta = bot.send_message(chat_id, 'Ingrese su cedula para completar la reserva:')

        bot.register_next_step_handler(pregunta, IngresoDeCedula)
    else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Volver al menu")
        pregunta = bot.send_message(chat_id, 'Seleccione o digite \"Menu\"', reply_markup=markup)


def IngresoDeCedula(message):
    chat_id = message.chat.id
    message_id = message.message_id
    vuelo = vuelo_dict[chat_id]

    cedula = message.text

    if not cedula.isdigit():
        pregunta = bot.send_message(chat_id, '¡Ingrese una cedula valida!')
        bot.register_next_step_handler(pregunta, IngresoDeCedula)
        return

    if not verificarCedula(cedula):
        pregunta = bot.send_message(chat_id, '¡Ingrese una cedula valida!')
        bot.register_next_step_handler(pregunta, IngresoDeCedula)
        return

    if vuelo.Retorno != "":
        reserva = vuelo.Origen + "|" + vuelo.Destino + "|" + str(
            int(vuelo.Adultos) + int(vuelo.Niños)) \
                  + "|" + str(vuelo.Salida) + "|" + str(
            vuelo.Retorno) + "|" + vuelo.HoraSalida + "|" + vuelo.HoraRetorno + "|" + vuelo.PrecioV
        for i in vuelo.Nombres:
            reserva=reserva + "|" + i
        vuelo.Reservas.append(reserva)
    else:
        reserva = vuelo.Origen + "|" + vuelo.Destino + "|" + str(
            int(vuelo.Adultos) + int(vuelo.Niños)) \
                  + "|" + str(vuelo.Salida) + "|" + vuelo.HoraSalida + "|" + vuelo.PrecioV
        for i in vuelo.Nombres:
            reserva=reserva + "|" + i
        vuelo.Reservas.append(reserva)

    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    markup.add("Volver al menu")
    pregunta = bot.send_message(chat_id, '¡Reserva exitosa!', reply_markup=markup)


def HoraRetorno(message):
    chat_id = message.chat.id
    message_id = message.message_id
    vuelo = vuelo_dict[chat_id]

    retorno = message.text
    valida = False
    if retorno.find("-") != -1:
        retornoT = retorno.replace("-", "").replace(":", "").replace(" ", "")
        if retornoT.isdigit() and len(retornoT) == 8:
            valida = True

    if valida == False:
        i = vuelo.Oferta
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)

        contador = 0
        for j in range(0, len(vuelo.EscalasR[i])):
            markup.add(vuelo.HorasR[i][contador] + " - " + vuelo.HorasR[i][contador + 1])
            contador = contador + 2

        pregunta = bot.send_message(chat_id, "Seleccione la hora de retorno:", reply_markup=markup)
        bot.register_next_step_handler(pregunta, HoraSalida)
        return

    vuelo.HoraRetorno = retorno

    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
    markup.add("Si", "No")
    pregunta = bot.send_message(chat_id, '¿Esta seguro de que quiere reservar el vuelo?', reply_markup=markup)

    bot.register_next_step_handler(pregunta, Reservar)


@bot.callback_query_handler(func=lambda call: True)
def cal(c):
    chat_id = c.message.chat.id
    vuelo = vuelo_dict[chat_id]
    if c.data.find("|") == -1:
        result, key, step = DetailedTelegramCalendar(min_date=date.today(),
                                                     max_date=date(date.today().year + 1, date.today().month,
                                                                   date.today().day)).process(c.data)

        if not result and key:
            if vuelo.Salida == "":
                bot.edit_message_text("Seleccione la fecha de salida:", c.message.chat.id, c.message.message_id,
                                      reply_markup=key)
            else:
                bot.edit_message_text("Seleccione la fecha de retorno:", c.message.chat.id, c.message.message_id,
                                      reply_markup=key)
        elif result:

            vuelo = vuelo_dict[chat_id]
            if vuelo.Tipo == "oneway":
                bot.send_message(chat_id, "Fecha de partida: " + str(result))

                markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
                markup.add("Economico", "Premium Economico", "Ejecutivo", "Primera Clase")
                pregunta = bot.send_message(chat_id, 'Seleccione en que clase desea viajar:', reply_markup=markup)

                vuelo.Salida = result
                bot.delete_message(chat_id, c.message.message_id)
                bot.register_next_step_handler(pregunta, Clase)

            else:
                if vuelo.Salida == "":
                    bot.delete_message(chat_id, c.message.message_id)
                    bot.send_message(chat_id, "Fecha de partida: " + str(result))
                    vuelo.Salida = result

                    calendar, step = DetailedTelegramCalendar(min_date=date.today(),
                                                              max_date=date(date.today().year, date.today().month,
                                                                            date.today().day)).build()
                    if vuelo.Salida == "":
                        bot.send_message(c.message.chat.id, "Seleccione la fecha de salida:", reply_markup=calendar)
                    else:
                        bot.send_message(c.message.chat.id, "Seleccione la fecha de retorno:", reply_markup=calendar)


                else:
                    fechaSalida = str(vuelo.Salida).split("-")
                    fechaRetorno = str(result).split("-")
                    validez = True

                    if int(fechaSalida[0]) > int(fechaRetorno[0]):
                        validez = False

                    if int(fechaSalida[1]) > int(fechaRetorno[1]):
                        validez = False

                    if int(fechaSalida[1]) == int(fechaRetorno[1]):
                        if int(fechaSalida[2]) > int(fechaRetorno[2]):
                            validez = False

                    if validez:
                        bot.send_message(chat_id, "Fecha de retorno: " + str(result))
                        vuelo.Retorno = result
                        bot.delete_message(chat_id, c.message.message_id)

                        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
                        markup.add("Economico", "Premium Economico", "Ejecutivo", "Primera Clase")
                        pregunta = bot.send_message(chat_id, 'Seleccione en que clase desea viajar:',
                                                    reply_markup=markup)

                        bot.register_next_step_handler(pregunta, Clase)
                    else:
                        bot.delete_message(chat_id, c.message.message_id)
                        bot.send_message(chat_id, '¡Seleccione un retono valido!')

                        calendar, step = DetailedTelegramCalendar(min_date=date.today(),
                                                                  max_date=date(date.today().year, date.today().month,
                                                                                date.today().day)).build()
                        if vuelo.Salida == "":
                            bot.send_message(c.message.chat.id, "Seleccione la fecha de salida:", reply_markup=calendar)
                        else:
                            bot.send_message(c.message.chat.id, "Seleccione la fecha de retorno:",
                                             reply_markup=calendar)

    elif(vuelo.Salida!=""):
        try:
            datos = c.data.split("|")
            i = int(datos[0])
            vuelo = vuelo_dict[chat_id]
            vuelo.Oferta = i
            vuelo.PrecioV = datos[1]
            for j in range(0, len(vuelo.Escalas) + 1):
                bot.delete_message(c.message.chat.id, c.message.message_id - j + len(vuelo.Escalas) - i - 1)

            message = "*\-\-\-\- Salida \-\-\-\-*\n"
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)

            contador = 0
            for j in range(0, len(vuelo.Escalas[i])):
                markup.add(vuelo.Horas[i][contador] + " - " + vuelo.Horas[i][contador + 1])
                message = message + "*" + vuelo.Origen + "* _" + vuelo.Horas[i][
                    contador] + "_ \- *" + vuelo.Destino + "* _" + \
                          vuelo.Horas[i][contador + 1] + "_\n*\- Duracion:* _" + vuelo.Duracion[i][
                              j] + "_ " + "*\n\- Escalas:* _" + vuelo.Escalas[i][j] + "_\n"
                contador = contador + 2

            if vuelo.Retorno != "":
                message = message + "*Retorno:*\n"
                contador = 0
                for j in range(0, len(vuelo.EscalasR[i])):
                    # markup.add(vuelo.HorasR[i][contador] + " - " + vuelo.HorasR[i][contador])
                    message = message + "*" + vuelo.Origen + "* _" + vuelo.HorasR[i][
                        contador] + "_ \- *" + vuelo.Destino + "* _" + \
                              vuelo.HorasR[i][contador + 1] + "_\n*\- Duracion:* _" + vuelo.DuracionR[i][
                                  j] + "_ " + "*\n\- Escalas:* _" + vuelo.EscalasR[i][j] + "_\n"
                    contador = contador + 2

            message=message+ "*Costo Final:* $" + vuelo.Precio[i]
            bot.send_message(chat_id, "Vuelo seleccionado:")
            bot.send_message(chat_id, message, parse_mode="MarkdownV2")
            pregunta = bot.send_message(chat_id, "Seleccione la hora de salida", reply_markup=markup)
            bot.register_next_step_handler(pregunta, HoraSalida)
        except Exception as e:
            print("Error: Boton en linea")


def verificarCedula(texto):
    if Verificacion_cedula:
        if len(texto) != 10:
            return False
        else:
            nocero = texto.strip("0")

            if nocero == "":
                return False

            cedula = int(nocero, 0)
            verificador = cedula % 10
            numero = cedula // 10

            # mientras tenga números
            suma = 0
            while (numero > 0):

                # posición impar
                posimpar = numero % 10
                numero = numero // 10
                posimpar = 2 * posimpar
                if (posimpar > 9):
                    posimpar = posimpar - 9

                # posición par
                pospar = numero % 10
                numero = numero // 10

                suma = suma + posimpar + pospar

            decenasup = suma // 10 + 1
            calculado = decenasup * 10 - suma
            if (calculado >= 10):
                calculado = calculado - 10

            if (calculado == verificador):
                return True
            else:
                return False
    else:
        return True


bot.enable_save_next_step_handlers(delay=2)
bot.load_next_step_handlers()
bot.set_update_listener(listener)
bot.polling()

while False:
    try:
        status = "Connected"
        bot.polling(none_stop=False, interval=1)
        pass

    except:
        status = "failure"

    else:
        print (status)

    time.sleep(1)



Ruben [1682499150]: /start
https://www.despegar.com.ec/shop/flights/results/oneway/GYE/CUE/2021-03-17/1/0/0/NA/NA/NA/NA?from=SB&di=1-0&reSearch=true
https://www.despegar.com.ec/shop/flights/results/oneway/GYE/CUE/2021-03-18/1/0/0/NA/NA/NA/NA?from=SB&di=1-0&reSearch=true
Ruben [1682499150]: Volver al menu
Ruben [1682499150]: Volver al menu
